In [18]:
import pandas as pd
import numpy as np
import datetime
import gymnasium as gym
from finrl.main import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.config import INDICATORS
from finrl.config_tickers import SP_500_TICKER
import pandas_datareader as web
from finrl.agents.stablebaselines3.models import DRLAgent
import itertools
from stable_baselines3.common.logger import configure
import pickle

In [3]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2022-07-01'
TRADE_START_DATE = '2022-07-01'
TRADE_END_DATE = '2024-05-01'


In [4]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                        end_date = TRADE_END_DATE,
                        ticker_list = SP_500_TICKER).fetch_data()

YF deprecation warning: set proxy via new config function: yf.set_config(proxy=proxy)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ABC']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ABMD']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (1631739, 8)


In [5]:
df_raw.to_csv('data_raw.csv', index=False)

In [6]:
df_raw = pd.read_csv('data_raw.csv')
df_raw['date'] = pd.to_datetime(df_raw['date'])

In [7]:
df_raw

,date,close,high,low,open,volume,tic,day
0,2009-01-02,10.320797,10.358929,9.856870,9.914066,4236220,A,4
1,2009-01-02,7.909602,7.994448,7.230828,7.287392,5167000,AAL,4
2,2009-01-02,29.373983,29.511647,28.453358,29.133073,795900,AAP,4
3,2009-01-02,2.724325,2.733032,2.556513,2.578127,746015200,AAPL,4
4,2009-01-02,17.518660,17.613515,17.129429,17.508848,13163193,ABT,4
...,...,...,...,...,...,...,...,...
1631734,2024-04-30,128.481766,131.440690,128.363807,129.808860,1397800,XYL,1
1631735,2024-04-30,137.190140,138.753865,135.733253,135.976068,4087300,YUM,1
1631736,2024-04-30,118.623993,119.738440,118.604272,119.265045,1429000,ZBH,1
1631737,2024-04-30,38.776276,39.441882,38.643155,38.804804,1429800,ZION,1


In [8]:
gdp = web.DataReader('GDP', 'fred', 2009, 2025)
gdp = gdp.reset_index()

In [9]:
date = pd.date_range(start=TRAIN_START_DATE, end=TRADE_END_DATE)
date_df = pd.DataFrame()
date_df['date'] = date

In [10]:
gdp['GDP_log'] = np.log(gdp['GDP'])
gdp.drop(columns=['GDP'], inplace=True)
gdp.head()

,DATE,GDP_log
0,2009-01-01,9.577127
1,2009-04-01,9.573680
2,2009-07-01,9.578372
3,2009-10-01,9.592281
4,2010-01-01,9.599988


In [11]:
gdp['date']=pd.to_datetime(gdp['DATE'])
gdp.rename(columns = {'GDP_log':'gdp_log'}, inplace = True)
gdp = gdp.drop(['DATE'], axis=1)
gdp_df=gdp.merge(date_df, on='date', how='right')
gdp_df = gdp_df.fillna(method='ffill')
gdp_df = gdp_df.merge(pd.DataFrame({"tic":df_raw.tic.unique()}),how="cross")
gdp_df.head()

/var/folders/zr/f_m9lv8d1px3nshyhcml84540000gn/T/ipykernel_94465/2012760736.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  gdp_df = gdp_df.fillna(method='ffill')


,gdp_log,date,tic
0,9.577127,2009-01-01,A
1,9.577127,2009-01-01,AAL
2,9.577127,2009-01-01,AAP
3,9.577127,2009-01-01,AAPL
4,9.577127,2009-01-01,ABT


In [12]:
df_merged = pd.merge(df_raw, gdp_df, how='left', left_on=['date', 'tic'], right_on=['date', 'tic'])
df_merged.head()

,date,close,high,low,open,volume,tic,day,gdp_log
0,2009-01-02,10.320797,10.358929,9.856870,9.914066,4236220,A,4,9.577127
1,2009-01-02,7.909602,7.994448,7.230828,7.287392,5167000,AAL,4,9.577127
2,2009-01-02,29.373983,29.511647,28.453358,29.133073,795900,AAP,4,9.577127
3,2009-01-02,2.724325,2.733032,2.556513,2.578127,746015200,AAPL,4,9.577127
4,2009-01-02,17.518660,17.613515,17.129429,17.508848,13163193,ABT,4,9.577127


In [13]:
df_raw.tic.value_counts()

tic
A       3857
NVDA    3857
NTRS    3857
NTAP    3857
NSC     3857
        ... 
IR      1753
FOXA    1294
DOW     1288
CTVA    1242
STI      502
Name: count, Length: 436, dtype: int64

In [14]:
fe = FeatureEngineer(use_technical_indicator=True,
                    tech_indicator_list=INDICATORS,
                    # use_vix=True,
                    use_turbulence=True,
                    user_defined_feature=True)

df = fe.preprocess_data(df_merged)
df = df.copy()
df = df.fillna(0)
df = df.replace(np.inf, 0)

Successfully added technical indicators


/opt/miniconda3/envs/summerresearch2025/lib/python3.11/site-packages/numpy/linalg/_linalg.py:3383: RuntimeWarning: divide by zero encountered in matmul
  return _core_matmul(x1, x2)
/opt/miniconda3/envs/summerresearch2025/lib/python3.11/site-packages/numpy/linalg/_linalg.py:3383: RuntimeWarning: overflow encountered in matmul
  return _core_matmul(x1, x2)
/opt/miniconda3/envs/summerresearch2025/lib/python3.11/site-packages/numpy/linalg/_linalg.py:3383: RuntimeWarning: invalid value encountered in matmul
  return _core_matmul(x1, x2)
/opt/miniconda3/envs/summerresearch2025/lib/python3.11/site-packages/numpy/linalg/_linalg.py:3383: RuntimeWarning: divide by zero encountered in matmul
  return _core_matmul(x1, x2)
/opt/miniconda3/envs/summerresearch2025/lib/python3.11/site-packages/numpy/linalg/_linalg.py:3383: RuntimeWarning: overflow encountered in matmul
  return _core_matmul(x1, x2)
/opt/miniconda3/envs/summerresearch2025/lib/python3.11/site-packages/numpy/linalg/_linalg.py:3383: Runt

Successfully added turbulence index
Successfully added user defined features


In [15]:
df

,date,close,high,low,open,volume,tic,day,gdp_log,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence,daily_return
0,2009-01-02,10.320797,10.358929,9.856870,9.914066,4236220,A,4,9.577127,0.000000,10.989876,10.001250,100.000000,66.666667,100.000000,10.320797,10.320797,0.00000,-0.233625
1,2009-01-02,7.909602,7.994448,7.230828,7.287392,5167000,AAL,4,9.577127,0.000000,10.989876,10.001250,100.000000,66.666667,100.000000,7.909602,7.909602,0.00000,-0.233625
2,2009-01-02,29.373983,29.511647,28.453358,29.133073,795900,AAP,4,9.577127,0.000000,10.989876,10.001250,100.000000,66.666667,100.000000,29.373983,29.373983,0.00000,2.713712
3,2009-01-02,2.724325,2.733032,2.556513,2.578127,746015200,AAPL,4,9.577127,0.000000,10.989876,10.001250,100.000000,66.666667,100.000000,2.724325,2.724325,0.00000,-0.907254
4,2009-01-02,17.518660,17.613515,17.129429,17.508848,13163193,ABT,4,9.577127,0.000000,10.989876,10.001250,100.000000,66.666667,100.000000,17.518660,17.518660,0.00000,5.430459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515796,2024-04-30,28.428505,28.788479,28.362194,28.513762,2331800,XRAY,1,10.280109,-0.581476,31.152030,27.924508,39.599903,-115.679678,35.231876,30.035300,30.754981,1479.49997,-0.746895
1515797,2024-04-30,11.917634,12.437741,11.908666,12.374970,4365900,XRX,1,10.280109,-0.818835,16.914388,11.834996,32.362435,-203.758534,59.553152,14.758122,15.347000,1479.49997,-0.580786
1515798,2024-04-30,137.190140,138.753865,135.733253,135.976068,4087300,YUM,1,10.280109,1.249690,139.005450,130.646753,58.326282,122.759275,29.476832,134.340796,132.964597,1479.49997,10.511525
1515799,2024-04-30,118.623993,119.738440,118.604272,119.265045,1429000,ZBH,1,10.280109,-1.759815,129.109546,114.627845,44.746912,-90.715938,12.762096,123.476827,123.640064,1479.49997,-0.135331


In [16]:
df_new = df.sort_values(['date', 'tic'], ignore_index=True)
df_new.index = df.date.factorize()[0]

cov_list = []
return_list = []

lookback = 252
for i in range(lookback, len(df_new.index.unique())):
    data_lookback = df_new.loc[i-lookback:i,:]
    price_lookback = data_lookback.pivot_table(index='date', columns='tic', values='close')
    return_lookback = price_lookback.pct_change().dropna()
    return_list.append(return_lookback)

    covs = return_lookback.cov().values
    cov_list.append(covs)

df_cov = pd.DataFrame({'date':df_new.date.unique()[lookback:], 'cov_list':cov_list, 'return_list':return_list})
df_new = df_new.merge(df_cov, on='date')
df_new = df_new.sort_values(['date', 'tic']).reset_index(drop=True)


In [17]:
train = data_split(df, TRAIN_START_DATE, TRAIN_END_DATE)
trade = data_split(df, TRADE_START_DATE, TRADE_END_DATE)
print(len(train))
print(len(trade))

1335021
180780


In [87]:
df_new.drop(columns=['daily_return'])

,date,close,high,low,open,volume,tic,day,gdp,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence,cov_list,return_list
0,2010-01-04,19.891680,20.101400,19.783641,19.948876,3815561,A,0,14764.610,0.385154,19.832911,18.381352,66.065015,219.629177,48.081007,18.934377,18.009170,0.000000,"[[0.0006651361957953325, 0.0007629625033154068...",tic A AAL AAP ...
1,2010-01-04,4.496877,4.657143,4.393175,4.562869,9837300,AAL,0,14764.610,0.264201,4.990098,3.970660,56.734695,56.087263,11.644350,4.087727,3.736085,0.000000,"[[0.0006651361957953325, 0.0007629625033154068...",tic A AAL AAP ...
2,2010-01-04,34.948494,35.519717,34.931184,35.225451,1701700,AAP,0,14764.610,0.260571,36.124009,34.439078,51.067335,19.989375,0.375261,34.985769,34.059244,0.000000,"[[0.0006651361957953325, 0.0007629625033154068...",tic A AAL AAP ...
3,2010-01-04,6.424604,6.439314,6.375671,6.407192,493729600,AAPL,0,14764.610,0.117870,6.487938,5.524102,62.133146,168.825967,33.760289,6.010478,5.957822,0.000000,"[[0.0006651361957953325, 0.0007629625033154068...",tic A AAL AAP ...
4,2010-01-04,18.414783,18.448598,18.232191,18.323488,10829095,ABT,0,14764.610,0.112347,18.481251,17.995303,59.540567,72.994924,13.002874,18.246506,17.839046,0.000000,"[[0.0006651361957953325, 0.0007629625033154068...",tic A AAL AAP ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1416760,2024-04-30,28.428505,28.788479,28.362194,28.513762,2331800,XRAY,1,29147.044,-0.581476,31.152029,27.924507,39.599938,-115.679608,35.231742,30.035300,30.754981,1479.528839,"[[0.00026970876861394274, 7.253527861140761e-0...",tic A AAL AAP ...
1416761,2024-04-30,11.917634,12.437741,11.908666,12.374970,4365900,XRX,1,29147.044,-0.818835,16.914389,11.834996,32.362459,-203.758525,59.553091,14.758122,15.347000,1479.528839,"[[0.00026970876861394274, 7.253527861140761e-0...",tic A AAL AAP ...
1416762,2024-04-30,137.190155,138.753881,135.733268,135.976083,4087300,YUM,1,29147.044,1.249691,139.005453,130.646755,58.326357,122.759773,29.477058,134.340797,132.964599,1479.528839,"[[0.00026970876861394274, 7.253527861140761e-0...",tic A AAL AAP ...
1416763,2024-04-30,118.623985,119.738432,118.604264,119.265038,1429000,ZBH,1,29147.044,-1.759816,129.109539,114.627847,44.746909,-90.716070,12.762126,123.476825,123.640064,1479.528839,"[[0.00026970876861394274, 7.253527861140761e-0...",tic A AAL AAP ...


In [23]:
train_pickle_path = 'data/train.pickle'
trade_pickle_path = 'data/trade.pickle'

with open(train_pickle_path, 'wb') as f:
    pickle.dump(train, f)

with open(trade_pickle_path, 'wb') as f:
    pickle.dump(trade, f)


In [103]:
stock_dimension = len(df_new.tic.unique())

In [149]:
state_space = 1 + 2*stock_dimension + len(INDICATORS) * stock_dimension + 1 * stock_dimension

In [150]:
train = data_split(df_new, TRAIN_START_DATE, TRAIN_END_DATE)

trade = data_split(df_new, TRADE_START_DATE, TRADE_END_DATE)

In [151]:
state_space

4324

In [152]:
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv

# Drop cov_list and return_list columns as StockTradingEnv doesn't use them

buy_cost_list = sell_cost_list = [0.005] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    'hmax':100,
    'initial_amount': 1000000,
    'num_stock_shares': num_stock_shares,
    'buy_cost_pct': buy_cost_list,
    'sell_cost_pct': sell_cost_list,
    'state_space': state_space,
    'stock_dim': stock_dimension,
    'tech_indicator_list': INDICATORS + ['gdp'],
    'action_space': stock_dimension,
    'reward_scaling': 1e-4
    
}

e_train_gym = StockTradingEnv(df=train, **env_kwargs)
env_train, _ = e_train_gym.get_sb_env()

In [158]:
e_train_gym.df.tic.count()

np.int64(1235985)

In [164]:
agent = DRLAgent(env = env_train)
model_ppo = agent.get_model('ppo')

tmp_path = RESULTS_DIR + '/ppo'
new_logger_ppo = configure(tmp_path, ['stdout', 'csv', 'tensorboard'])

model_ppo.set_logger(new_logger_ppo)

trained_ppo = agent.train_model(model=model_ppo,
                                tb_log_name='ppo',
                                total_timesteps=50000)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 64}
Using cuda device
Logging to results/ppo
-------------------------------------
| time/              |              |
|    fps             | 13           |
|    iterations      | 1            |
|    time_elapsed    | 147          |
|    total_timesteps | 2048         |
| train/             |              |
|    reward          | -0.027009655 |
-------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 13         |
|    iterations           | 2          |
|    time_elapsed         | 296        |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.07965393 |
|    clip_fraction        | 0.63       |
|    clip_range           | 0.2        |
|    entropy_loss         | -559       |
|    explained_variance   | -0.00297   |
|    learning_rate        | 0.00025    |
|   